# Supervised Machine Learning - Balance of actors classifier
   
In this notebook, the manual content analysis data is used to train and evaluate a classifier that assesses the balance of actors of an article.   
The process includes feature selection, and the evaluation and comparison of different types of classifiers on different types of text representations.

## Load packages

In [4]:
#import relevant packages
import pandas as pd
from pandas import read_excel
import re
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, ShuffleSplit, GridSearchCV, train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pprint import pprint
import joblib

## Read data

In [5]:
#read in the manually coded data
df = read_excel("mca_cleaned.xlsx")

In [6]:
#remove unreliable coders
df = df[df.CID != 4]
len(df)

449

## Splitting the data into test and training set

In [7]:
#inspect the category distribution to see how balanced it is
df["BOA"].value_counts()

0    297
1    152
Name: BOA, dtype: int64

In [8]:
#create training and testing dataset 
x_train, x_test, y_train, y_test = train_test_split(df["clean text"], df.BOA, test_size=0.2, random_state=1)

# Feature engineering
   
Four different types of text representations for the classifier training were used, namely count vectors, TF-IDF vectors with unigrams, and TF-IDF vectors with bigrams and TF-IDF vectors with both uni- and bigrams. All classifiers were trained and tested on all features.   
First, the labels, which remain in a binary format, are renamed. After that, the different vector types are created. 

In [9]:
labels_train = y_train
labels_test = y_test

### Count Vectors

In [10]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer="word", 
                             token_pattern=r"\w{1,10}", 
                             min_df = 10, 
                             max_df = 1., 
                             max_features=200)

#features for training
features_train_count = count_vect.fit_transform(x_train).toarray()

#features for testing
features_test_count = count_vect.fit_transform(x_test).toarray()

#inspect the shape
print(features_train_count.shape)
print(features_test_count.shape)

(359, 200)
(90, 200)


### TF-IDF Vectors
In the following, Term Frequency-Inverse Document Frequency is applied in order to represent the text data as a vector that can be used as numerical input for a SML algorithm. 
Three different vectors are created:
    - A vector with unigrams only
    - A vector with bigrams only
    - A vector with unigrams & bigrams
    
In addition to that, several parameters were specified:
    - Terms that appear in less than 10 documents are ignored -> min_df
    - All other terms are included -> max_df
    - In total, up to 200 features can be extracted per text -> max_features

In [14]:
# unigrams
tfidf_vect_ug = TfidfVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}',
                             ngram_range = (1,1),
                             min_df = 10, 
                             max_df = 1., 
                             max_features=200)

# bigrams
tfidf_vect_bg = TfidfVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}',
                             ngram_range = (2,2),
                             min_df = 10, 
                             max_df = 1., 
                             max_features=5) #otherwise there is an error

# unigrams and bigrams
tfidf_vect_ubg = TfidfVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}',
                             ngram_range = (1,2),
                             min_df = 10, 
                             max_df = 1., 
                             max_features=200)


#features for testing
features_train_tfidf_ug = tfidf_vect_ug.fit_transform(x_train).toarray()
features_train_tfidf_bg = tfidf_vect_bg.fit_transform(x_train).toarray()
features_train_tfidf_ubg = tfidf_vect_ubg.fit_transform(x_train).toarray()

#features (=y_train)
features_test_tfidf_ug = tfidf_vect_ug.fit_transform(x_test).toarray()
features_test_tfidf_bg = tfidf_vect_bg.fit_transform(x_test).toarray()
features_test_tfidf_ubg = tfidf_vect_ubg.fit_transform(x_test).toarray()

#Explore the shape of the features
print(features_train_tfidf_ubg.shape)
print(features_test_tfidf_ubg.shape)

(359, 200)
(90, 200)


## Model training & evaluation
In the following, different classifiers are trained and evaluated through their precision, recall and accuracy.

In [15]:
def train_model(classifier, features_train, labels_train, features_test):
    # fit the training dataset on the classifier
    classifier.fit(features_train, labels_train)
    # predict the labels on validation dataset
    predictions = classifier.predict(features_test)
    #calculate the accuracy of the predictions
    accuracy = classification_report(labels_test, predictions)
    print(accuracy)

### Stochastic Gradient Descent Classifier
First, the classifiers for different vectors are trained and evaluated on the training and test data.

In [19]:
#defining the classifier
sgdc = SGDClassifier(loss="hinge", max_iter=200, random_state=8) 
#training and evaluating the classifier
train_model(sgdc, features_train_count, labels_train, features_test_count)
train_model(sgdc, features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(sgdc, features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(sgdc, features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.74      0.54      0.62        63
           1       0.34      0.56      0.42        27

    accuracy                           0.54        90
   macro avg       0.54      0.55      0.52        90
weighted avg       0.62      0.54      0.56        90

              precision    recall  f1-score   support

           0       0.68      0.60      0.64        63
           1       0.26      0.33      0.30        27

    accuracy                           0.52        90
   macro avg       0.47      0.47      0.47        90
weighted avg       0.55      0.52      0.54        90

              precision    recall  f1-score   support

           0       0.75      0.10      0.17        63
           1       0.30      0.93      0.46        27

    accuracy                           0.34        90
   macro avg       0.53      0.51      0.31        90
weighted avg       0.62      0.34      0.26        90

              preci

#### Hyperparameter tuning & cross validation
Second, a grid search is performed in order to establish and cross-validate the best model parameters

In [20]:
def cross_validate(classifier, features_train, labels_train, cv):
    scores = cross_val_score(classifier, features_train, labels_train, cv=cv, scoring ="f1")
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = SGDClassifier(loss="hinge", max_iter=200, random_state=8)
calibrated_clf = CalibratedClassifierCV(base_estimator=clf, method="sigmoid", cv=10)  
grid_params = {"base_estimator__alpha": [0.0001, 0.001, 0.01, 0.1]}  
grid_search = GridSearchCV(estimator=calibrated_clf, param_grid=grid_params, cv=10)
grid_search.fit(features_train_tfidf_bg, labels_train)
print("Count vectors:", grid_search.best_params_)
print(cross_validate(sgdc, features_train_count, labels_train, 10))
grid_search.fit(features_train_count, labels_train)
print("TF-IDF vectors with unigrams:", grid_search.best_params_)
print(cross_validate(sgdc, features_train_tfidf_ug, labels_train, 10))
grid_search.fit(features_train_tfidf_ug, labels_train)
print("TF-IDF vectors with bigrams:", grid_search.best_params_)
print(cross_validate(sgdc, features_train_tfidf_ug, labels_train, 10))
grid_search.fit(features_train_tfidf_bg, labels_train)
print("TF-IDF vectors with uni- and bigrams:",grid_search.best_params_)
print(cross_validate(sgdc, features_train_tfidf_ubg, labels_train, 10))

Count vectors: {'base_estimator__alpha': 0.1}
Accuracy: 0.46 (+/- 0.18)
None
TF-IDF vectors with unigrams: {'base_estimator__alpha': 0.01}
Accuracy: 0.46 (+/- 0.15)
None
TF-IDF vectors with bigrams: {'base_estimator__alpha': 0.1}
Accuracy: 0.46 (+/- 0.15)
None
TF-IDF vectors with uni- and bigrams: {'base_estimator__alpha': 0.1}
Accuracy: 0.44 (+/- 0.10)
None


In [21]:
#incorporating the grid-search results for training and evaluating the classifiers for different vectors
train_model(SGDClassifier(loss="hinge", alpha = .1, max_iter=200, random_state=8), features_train_count, labels_train, features_test_count)
train_model(SGDClassifier(loss="hinge", alpha = .01, max_iter=200, random_state=8), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(SGDClassifier(loss="hinge", alpha = .1, max_iter=200, random_state=8), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(SGDClassifier(loss="hinge", alpha = .1, max_iter=200, random_state=8), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.71      0.65      0.68        63
           1       0.31      0.37      0.34        27

    accuracy                           0.57        90
   macro avg       0.51      0.51      0.51        90
weighted avg       0.59      0.57      0.58        90

              precision    recall  f1-score   support

           0       0.70      1.00      0.82        63
           1       0.00      0.00      0.00        27

    accuracy                           0.70        90
   macro avg       0.35      0.50      0.41        90
weighted avg       0.49      0.70      0.58        90

              precision    recall  f1-score   support

           0       0.70      1.00      0.82        63
           1       0.00      0.00      0.00        27

    accuracy                           0.70        90
   macro avg       0.35      0.50      0.41        90
weighted avg       0.49      0.70      0.58        90

              preci

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Save the best classification results for later comparison

In [22]:
clf = SGDClassifier(loss="hinge", alpha = .1, max_iter=200, random_state=8)
clf.fit(features_train_count, labels_train)
sgdc_final = clf.predict(features_test_count)

### Naive Bayes Classifier
First, the classifiers for different vectors are trained and evaluated on the training and test data.

In [23]:
# Naive Bayes on Count Vectors
print("Gaussian NB")
train_model(GaussianNB(), features_train_count, labels_train, features_test_count)
train_model(GaussianNB(), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(GaussianNB(), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(GaussianNB(), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

print("")
print("")

print("Multinomial NB")
train_model(MultinomialNB(), features_train_count, labels_train, features_test_count)
train_model(MultinomialNB(), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(MultinomialNB(), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(MultinomialNB(), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

Gaussian NB
              precision    recall  f1-score   support

           0       0.72      0.52      0.61        63
           1       0.32      0.52      0.39        27

    accuracy                           0.52        90
   macro avg       0.52      0.52      0.50        90
weighted avg       0.60      0.52      0.54        90

              precision    recall  f1-score   support

           0       0.70      0.73      0.71        63
           1       0.29      0.26      0.27        27

    accuracy                           0.59        90
   macro avg       0.49      0.49      0.49        90
weighted avg       0.58      0.59      0.58        90

              precision    recall  f1-score   support

           0       0.72      0.92      0.81        63
           1       0.44      0.15      0.22        27

    accuracy                           0.69        90
   macro avg       0.58      0.53      0.51        90
weighted avg       0.63      0.69      0.63        90

       

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Hyperparameter tuning
Second, a grid search is performed in order to establish and cross-validate the best model parameters. 

In [24]:
clf = MultinomialNB()
calibrated_clf = CalibratedClassifierCV(base_estimator=clf, method="sigmoid", cv=10)  
grid_params = {"base_estimator__alpha": [0.0001, 0.001, 0.01, 0.1]}  
grid_search = GridSearchCV(estimator=calibrated_clf, param_grid=grid_params, cv=10)
grid_search.fit(features_train_tfidf_bg, labels_train)
print("Count vectors:", grid_search.best_params_)
print(cross_validate(sgdc, features_train_count, labels_train, 10))
grid_search.fit(features_train_count, labels_train)
print("TF-IDF vectors with unigrams:", grid_search.best_params_)
print(cross_validate(sgdc, features_train_tfidf_ug, labels_train, 10))
grid_search.fit(features_train_tfidf_ug, labels_train)
print("TF-IDF vectors with bigrams:", grid_search.best_params_)
print(cross_validate(sgdc, features_train_tfidf_ug, labels_train, 10))
grid_search.fit(features_train_tfidf_bg, labels_train)
print("TF-IDF vectors with uni- and bigrams:",grid_search.best_params_)
print(cross_validate(sgdc, features_train_tfidf_ubg, labels_train, 10))

Count vectors: {'base_estimator__alpha': 0.0001}
Accuracy: 0.46 (+/- 0.18)
None
TF-IDF vectors with unigrams: {'base_estimator__alpha': 0.0001}
Accuracy: 0.46 (+/- 0.15)
None
TF-IDF vectors with bigrams: {'base_estimator__alpha': 0.1}
Accuracy: 0.46 (+/- 0.15)
None
TF-IDF vectors with uni- and bigrams: {'base_estimator__alpha': 0.0001}
Accuracy: 0.44 (+/- 0.10)
None


In [25]:
#incorporating the grid-search results for training and evaluating the classifiers for different vectors
train_model(MultinomialNB(alpha = .0001), features_train_count, labels_train, features_test_count)
train_model(MultinomialNB(alpha = .0001), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(MultinomialNB(alpha = .1,), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(MultinomialNB(alpha = .0001), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.62      0.32      0.42        63
           1       0.26      0.56      0.35        27

    accuracy                           0.39        90
   macro avg       0.44      0.44      0.39        90
weighted avg       0.52      0.39      0.40        90

              precision    recall  f1-score   support

           0       0.71      1.00      0.83        63
           1       1.00      0.04      0.07        27

    accuracy                           0.71        90
   macro avg       0.85      0.52      0.45        90
weighted avg       0.80      0.71      0.60        90

              precision    recall  f1-score   support

           0       0.70      0.98      0.82        63
           1       0.00      0.00      0.00        27

    accuracy                           0.69        90
   macro avg       0.35      0.49      0.41        90
weighted avg       0.49      0.69      0.57        90

              preci

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#compare results with the best Gaussian classifier
train_model(GaussianNB(), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.74      0.78      0.76        63
           1       0.42      0.37      0.39        27

    accuracy                           0.66        90
   macro avg       0.58      0.57      0.58        90
weighted avg       0.64      0.66      0.65        90



### Save best classifier for later comparison

In [27]:
clf = GaussianNB()
clf.fit(features_train_tfidf_ubg, labels_train)
nbc_final = clf.predict(features_test_tfidf_ubg)

## Support Vector Machines
First, the classifiers for different vectors are trained and evaluated on the training and test data.

In [28]:
train_model(svm.SVC(random_state=8), features_train_count, labels_train, features_test_count)
train_model(svm.SVC(random_state=8), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(svm.SVC(random_state=8), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(svm.SVC(random_state=8), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.72      0.95      0.82        63
           1       0.57      0.15      0.24        27

    accuracy                           0.71        90
   macro avg       0.65      0.55      0.53        90
weighted avg       0.68      0.71      0.65        90

              precision    recall  f1-score   support

           0       0.72      1.00      0.83        63
           1       1.00      0.07      0.14        27

    accuracy                           0.72        90
   macro avg       0.86      0.54      0.49        90
weighted avg       0.80      0.72      0.63        90

              precision    recall  f1-score   support

           0       0.70      0.94      0.80        63
           1       0.33      0.07      0.12        27

    accuracy                           0.68        90
   macro avg       0.52      0.51      0.46        90
weighted avg       0.59      0.68      0.60        90

              preci

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Hyperparameter tuning
Second, a grid search is performed in order to establish and cross-validate the best model parameters. 

In [29]:
# C
C = [.0001, .001, .01]

# gamma
gamma = [.0001, .001, .01, .1, 1, 10, 100]

# degree
degree = [1, 2, 3, 4, 5]

# kernel
kernel = ['linear', 'rbf', 'poly']

# probability
probability = [True]

# Create the random grid
random_grid = {'C': C,
              'kernel': kernel,
              'gamma': gamma,
              'degree': degree,
              'probability': probability
             }

In [30]:
# First create the base model to tune
svc = svm.SVC(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=svc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='f1',
                                   cv=10, 
                                   verbose=1, 
                                   random_state=8)

In [31]:
random_search.fit(features_train_count, labels_train)

print("The best hyperparameters from Random Search for count vectors are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The best hyperparameters from Random Search for count vectors are:
{'probability': True, 'kernel': 'poly', 'gamma': 10, 'degree': 4, 'C': 0.01}

The mean accuracy of a model with these hyperparameters is:
0.5101051972418034


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.2min finished


In [32]:
random_search.fit(features_train_tfidf_ug, labels_train)

print("The best hyperparameters from Random Search for tfidf vectors with unigrams are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The best hyperparameters from Random Search for tfidf vectors with unigrams are:
{'probability': True, 'kernel': 'poly', 'gamma': 10, 'degree': 4, 'C': 0.01}

The mean accuracy of a model with these hyperparameters is:
0.1534938590820944


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.1min finished


In [33]:
random_search.fit(features_train_tfidf_bg, labels_train)

print("The best hyperparameters from Random Search for tfidf vectors with bigrams are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    8.3s finished


The best hyperparameters from Random Search for tfidf vectors with bigrams are:
{'probability': True, 'kernel': 'poly', 'gamma': 100, 'degree': 4, 'C': 0.01}

The mean accuracy of a model with these hyperparameters is:
0.1742150247413405


In [34]:
random_search.fit(features_train_tfidf_ubg, labels_train)

print("The best hyperparameters from Random Search for tfidf vectors with uni and bigrams are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The best hyperparameters from Random Search for tfidf vectors with uni and bigrams are:
{'probability': True, 'kernel': 'poly', 'gamma': 10, 'degree': 4, 'C': 0.01}

The mean accuracy of a model with these hyperparameters is:
0.1534938590820944


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.1min finished


In [35]:
#incorporate grid_search results
train_model(svm.SVC(random_state=8, probability= True, kernel= 'poly', gamma= 10, degree= 4, C= 0.01), features_train_count, labels_train, features_test_count)
train_model(svm.SVC(random_state=8, probability= True, kernel= 'poly', gamma= 10, degree= 4, C= 0.01), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(svm.SVC(random_state=8, probability= True, kernel= 'poly', gamma= 100, degree= 4, C= 0.01), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(svm.SVC(random_state=8, probability= True, kernel= 'poly', gamma= 10, degree= 4, C= 0.01), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.74      0.78      0.76        63
           1       0.42      0.37      0.39        27

    accuracy                           0.66        90
   macro avg       0.58      0.57      0.58        90
weighted avg       0.64      0.66      0.65        90



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.70      1.00      0.82        63
           1       0.00      0.00      0.00        27

    accuracy                           0.70        90
   macro avg       0.35      0.50      0.41        90
weighted avg       0.49      0.70      0.58        90

              precision    recall  f1-score   support

           0       0.70      0.92      0.79        63
           1       0.29      0.07      0.12        27

    accuracy                           0.67        90
   macro avg       0.49      0.50      0.46        90
weighted avg       0.57      0.67      0.59        90

              precision    recall  f1-score   support

           0       0.70      1.00      0.82        63
           1       0.00      0.00      0.00        27

    accuracy                           0.70        90
   macro avg       0.35      0.50      0.41        90
weighted avg       0.49      0.70      0.58        90



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
#compare it to the base model for the vectors that achieved the best results
base_model = svm.SVC()
base_model.fit(features_train_count, labels_train)

svc_pred = base_model.predict(features_test_count)
print(classification_report(labels_test, svc_pred))

              precision    recall  f1-score   support

           0       0.72      0.95      0.82        63
           1       0.57      0.15      0.24        27

    accuracy                           0.71        90
   macro avg       0.65      0.55      0.53        90
weighted avg       0.68      0.71      0.65        90



### Save the best classification results for later comparison

In [38]:
clf = svm.SVC(random_state=8, probability= True, kernel= 'poly', gamma= 10, degree= 4, C= 0.01)
clf.fit(features_train_count, labels_train)
svc_final = clf.predict(features_test_count)

## K-nearest neighbour classifier

In [39]:
train_model(KNeighborsClassifier(), features_train_count, labels_train, features_test_count)
train_model(KNeighborsClassifier(), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(KNeighborsClassifier(), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(KNeighborsClassifier(), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.72      0.92      0.81        63
           1       0.44      0.15      0.22        27

    accuracy                           0.69        90
   macro avg       0.58      0.53      0.51        90
weighted avg       0.63      0.69      0.63        90

              precision    recall  f1-score   support

           0       0.71      0.63      0.67        63
           1       0.32      0.41      0.36        27

    accuracy                           0.57        90
   macro avg       0.52      0.52      0.52        90
weighted avg       0.60      0.57      0.58        90

              precision    recall  f1-score   support

           0       0.71      0.95      0.81        63
           1       0.40      0.07      0.12        27

    accuracy                           0.69        90
   macro avg       0.55      0.51      0.47        90
weighted avg       0.61      0.69      0.61        90

              preci

## Cross-Validation for Hyperparameter tuning

### Grid Search Cross Validation

In [42]:
# Create the parameter grid 
n_neighbors = [int(x) for x in np.linspace(start = 1, stop = 250, num = 200)]

param_grid = {"n_neighbors": n_neighbors}

# Create a base model
knnc = KNeighborsClassifier()

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .2, random_state = 8)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=knnc, 
                           param_grid=param_grid,
                           scoring="f1",
                           cv=cv_sets,
                           verbose=1)

In [43]:
grid_search.fit(features_train_count, labels_train)

print("The best hyperparameters from Grid Search for count vectors are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The best hyperparameters from Grid Search for count vectors are:
{'n_neighbors': 3}

The mean accuracy of a model with these hyperparameters is:
0.43412698412698414


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    8.0s finished


In [44]:
grid_search.fit(features_train_tfidf_ug, labels_train)

print("The best hyperparameters from Grid Search for tfidf vectors are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 200 candidates, totalling 600 fits
The best hyperparameters from Grid Search for tfidf vectors are:
{'n_neighbors': 7}

The mean accuracy of a model with these hyperparameters is:
0.4282682770077728


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    8.1s finished


In [45]:
grid_search.fit(features_train_tfidf_bg, labels_train)

print("The best hyperparameters from Grid Search for tfidf vectors with ngrams are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 200 candidates, totalling 600 fits
The best hyperparameters from Grid Search for tfidf vectors with ngrams are:
{'n_neighbors': 1}

The mean accuracy of a model with these hyperparameters is:
0.39477124183006534


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    3.4s finished


In [46]:
grid_search.fit(features_train_tfidf_ubg, labels_train)

print("The best hyperparameters from Grid Search for tfidf vectors with ngrams are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 200 candidates, totalling 600 fits
The best hyperparameters from Grid Search for tfidf vectors with ngrams are:
{'n_neighbors': 7}

The mean accuracy of a model with these hyperparameters is:
0.4272249060654858


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    8.1s finished


In [47]:
#incorporating the grid-search results for training and evaluating the classifiers for different vectors
train_model(KNeighborsClassifier(n_neighbors= 3), features_train_count, labels_train, features_test_count)
train_model(KNeighborsClassifier(n_neighbors= 7), features_train_tfidf_ug, labels_train, features_test_tfidf_ug)
train_model(KNeighborsClassifier(n_neighbors= 1), features_train_tfidf_bg, labels_train, features_test_tfidf_bg)
train_model(KNeighborsClassifier(n_neighbors= 7), features_train_tfidf_ubg, labels_train, features_test_tfidf_ubg)

              precision    recall  f1-score   support

           0       0.72      0.94      0.81        63
           1       0.50      0.15      0.23        27

    accuracy                           0.70        90
   macro avg       0.61      0.54      0.52        90
weighted avg       0.65      0.70      0.64        90

              precision    recall  f1-score   support

           0       0.72      0.76      0.74        63
           1       0.35      0.30      0.32        27

    accuracy                           0.62        90
   macro avg       0.53      0.53      0.53        90
weighted avg       0.61      0.62      0.61        90

              precision    recall  f1-score   support

           0       0.66      0.37      0.47        63
           1       0.27      0.56      0.37        27

    accuracy                           0.42        90
   macro avg       0.46      0.46      0.42        90
weighted avg       0.54      0.42      0.44        90

              preci

In [48]:
#compare results to the best base model
base_model = KNeighborsClassifier()
base_model.fit(features_train_tfidf_ubg, labels_train)

base_model_pred = base_model.predict(features_test_tfidf_ubg)
print(classification_report(labels_test, base_model_pred))

              precision    recall  f1-score   support

           0       0.71      0.63      0.67        63
           1       0.32      0.41      0.36        27

    accuracy                           0.57        90
   macro avg       0.52      0.52      0.52        90
weighted avg       0.60      0.57      0.58        90



### Save the best classification results for later comparison

In [49]:
clf = KNeighborsClassifier(n_neighbors= 7)
clf.fit(features_train_tfidf_ug, labels_train)
knn_final = clf.predict(features_test_tfidf_ug)

## Final Model comparison

In [50]:
print("classification report - Stochastic Gradiend Descent") 
print(classification_report(labels_test, sgdc_final))
print("classification report - Naive Bayes") 
print(classification_report(labels_test, nbc_final))
print("classification report - SVC") 
print(classification_report(labels_test, svc_final))
print("classification report - k nearest neighbour") 
print(classification_report(labels_test, knn_final))

classification report - Stochastic Gradiend Descent
              precision    recall  f1-score   support

           0       0.71      0.65      0.68        63
           1       0.31      0.37      0.34        27

    accuracy                           0.57        90
   macro avg       0.51      0.51      0.51        90
weighted avg       0.59      0.57      0.58        90

classification report - Naive Bayes
              precision    recall  f1-score   support

           0       0.74      0.78      0.76        63
           1       0.42      0.37      0.39        27

    accuracy                           0.66        90
   macro avg       0.58      0.57      0.58        90
weighted avg       0.64      0.66      0.65        90

classification report - SVC
              precision    recall  f1-score   support

           0       0.74      0.78      0.76        63
           1       0.42      0.37      0.39        27

    accuracy                           0.66        90
   macro avg